### Notebook for trading_bot

Starting with paper trading

In [34]:
import sys
import os

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "../../"))
sys.path.insert(0, BASE_DIR)

import lumibot
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from datetime import datetime

import math

# News
from alpaca_trade_api import REST
from timedelta import Timedelta

# Strategies
from strategies.ML_models.finbert_utils import estimate_sentiment

c:\Users\Victohu1\Desktop\algo_trading\algo_trading\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [10]:
API_KEY = "PK5VOHYTOFZB5OCY1CDE"
API_SECRET = "NxTEBhwXnCoKOeVXQ5TrWWQNRxiv3jU4UUyAZ8Y5"
BASE_URL = "https://paper-api.alpaca.markets/v2"

In [11]:
ALPACA_CREDS = {
    "API_KEY": API_KEY,
    "API_SECRET": API_SECRET,
    "PAPER": True
}

In [26]:
class MLTrader(Strategy):
    def initialize(self, symbol:str="SPY", cash_at_risk:float=.5):
        self.symbol= symbol
        self.sleeptime = "24H" # dictates how frequently we're gonna trade
        self.last_trade = None
        self.cash_at_risk = cash_at_risk # Cash at risk: How much of our cash balance do we want to risk at every trade, 0.5 means for each trade we're using 50% of our remaining cash balance
        self.api = REST(base_url=BASE_URL, key_id=API_KEY, secret_key=API_SECRET)
    
    def position_sizing(self):
        cash = self.get_cash()
        last_price = self.get_last_price(self.symbol)
        
        # Calculating sizing
        quantity = math.floor(cash * self.cash_at_risk / last_price) # tells us how many units we're gonna get per amount that we want to risk
        return cash, last_price, quantity

    def get_dates(self):
        today = self.get_datetime()

        # Get 3 days worth of news
        three_days_prior = today - Timedelta(days=3)
        return today.strftime('%Y-%m-%d'), three_days_prior.strftime('%Y-%m-%d')

    def get_news(self):
        today, three_days_prior = self.get_dates()
        news = self.api.get_news(symbol=self.symbol, 
                                 start=three_days_prior, 
                                 end=today)
        news = [ev.__dict__["_raw"]["headline"] for ev in news] # ev: event
        return news

    def on_trading_iteration(self): # runs once everything we get a tick
        cash, last_price, quantity = self.position_sizing()

        if cash > last_price: # check to make sure we're not buying when we don't have enough cash
            # Baseline trade
            if self.last_trade == None:
                news = self.get_news()
                print(news)
                order = self.create_order(
                    self.symbol,
                    quantity,
                    "buy",
                    type="bracket",

                    # stop loss
                    take_profit_price=last_price*1.2,
                    stop_loss_price=last_price*0.95
                )
                self.submit_order(order)
                self.last_trade = "buy"

In [28]:
start_date = datetime(2023, 12, 15)
end_date = datetime(2023, 12, 31)

In [29]:
broker = Alpaca(ALPACA_CREDS)
strategy = MLTrader(name='mlstrat', broker=broker,
                    parameters={"symbol":"SPY",
                                "cash_at_risk":.5})

In [30]:
strategy.backtest(
    YahooDataBacktesting,
    start_date,
    end_date,
    parameters={"symbol":"SPY", "cash_at_risk":.5}
)

Progress ||   2.47%  [Elapsed: 0:00:00 ETA: 0:00:18] ["If You Invested $1,000 In Exxon Mobil When Joe Biden Was Elected President, Here's How Much You'd Have Today", 'U.S. Senators Will Vote On Foreign Aid Supplemental Agreement Next Week', 'Traders May Need Frequent Portfolio Adjustments In 2024: BlackRock', 'U.S. Supreme Court Declines To Block Illinois Ban On Assault-Type Rifles And Large Capacity Magazines', "US Stocks Eye 7-Day Winning Streak As Mortgage Rates Fall Below 7%, Yields Tumble: What's Driving Markets Thursday?", 'Live On CNBC, SEC Chair Gary Gensler Discusses Bitcoin ETFs; Regulating Crypto', "Live On CNBC, SEC Chair Gary Gensler Discusses The SEC's Latest Moves; Treasury Clearing; Regulating Artificial Intelligence", "U.S. House Of Representatives Passes $886B Defense Policy Bill 310 To 118, Sending Measure To White House For Biden's Signature", 'Market Clubhouse Morning Memo - December 14th, 2023 (Trade Strategy For SPY, QQQ, AAPL, MSFT, TSLA, NVDA, GOOGL, And META)'

{'cagr': 0.25708368926531966,
 'volatility': 0.028893175680975387,
 'sharpe': 7.093498247517957,
 'max_drawdown': {'drawdown': 0.0027690901105407078,
  'date': Timestamp('2023-12-21 09:30:00-0500', tz='America/New_York')},
 'romad': 92.84049236488013,
 'total_return': 0.00944036000000037}